In [1]:
!pip install -e visualization/ -q

In [3]:
import os
from pathlib import Path
import shutil
from typing import Optional
from visualization.visualizer import draw_frame, get_frame_info, create_video, get_unique_frame_ids
import cv2
import json
from tqdm import tqdm
from visualization.visualizer import FrameInfo


json_files = os.listdir("data/gamestate_output")

referee_detected_videos = []
all_videos = []
for json_file in json_files:
    json_file = f"data/gamestate_output/{json_file}"
    if "action" not in json_file:
        continue

    # Check if file exists
    if not os.path.exists(json_file):
        msg = f"Error: JSON file '{json_file}' not found."
        raise ValueError(msg)

    # Get filename without extension for output folder
    filename = Path(json_file).stem

    # Get unique frame IDs
    frame_ids = get_unique_frame_ids(json_file)

    if not frame_ids:
        msg = "Error: No frames found in the JSON file"
        raise ValueError(msg)

    video_info = {}
    for frame_id in frame_ids:
        frame_info = get_frame_info(json_file, frame_id)
        video_info[frame_id] = frame_info.model_dump()

    os.makedirs("data/processed_gamestate_output", exist_ok=True)
    with open("data/processed_gamestate_output/" + filename + ".json", "w") as f:
        json.dump(video_info, f)

    # Check if referee is detected
    referee_detected = False
    for frame_id, frame_info in video_info.items():
        frame_info = FrameInfo.model_validate(frame_info)
        for player in frame_info.players:
            if player.role == "referee":
                referee_detected = True
                break
    
    if referee_detected:
        referee_detected_videos.append(filename)
    all_videos.append(filename)

In [23]:
import os
import json
from visualization.visualizer import FrameInfo

processed_dir = "data/processed_gamestate_output"

referee_detected_videos = []
all_videos = []
for file in os.listdir(processed_dir):
    if not file.endswith(".json"):
        continue
    action_name = file.split(".json")[0]
    with open(os.path.join(processed_dir, file), "r") as f:
        video_info = json.load(f)
    
    # Check if referee is detected
    referee_detected = False
    for frame_id, frame_info in video_info.items():
        frame_info = FrameInfo.model_validate(frame_info)
        for player in frame_info.players:
            if player.role == "referee":
                referee_detected = True
                break
    
    if referee_detected:
        referee_detected_videos.append(action_name)
    all_videos.append(action_name)


In [24]:
print(len(referee_detected_videos))
print(len(all_videos))

224
400


In [25]:
print(referee_detected_videos)

with open("referee_detected_videos.json", "w") as f:
    json.dump(referee_detected_videos, f)

['action_2230', 'action_2725', 'action_1670', 'action_1959', 'action_2226', 'action_637', 'action_2508', 'action_1975', 'action_2089', 'action_477', 'action_2748', 'action_1871', 'action_1037', 'action_573', 'action_1963', 'action_748', 'action_461', 'action_2836', 'action_2904', 'action_2841', 'action_504', 'action_1943', 'action_2683', 'action_1390', 'action_1685', 'action_1113', 'action_2857', 'action_2085', 'action_2590', 'action_2656', 'action_1304', 'action_2197', 'action_1611', 'action_1492', 'action_2752', 'action_713', 'action_2705', 'action_2354', 'action_2092', 'action_2568', 'action_1485', 'action_36', 'action_20', 'action_704', 'action_587', 'action_568', 'action_1581', 'action_1407', 'action_1542', 'action_769', 'action_2397', 'action_1104', 'action_2285', 'action_964', 'action_1523', 'action_2749', 'action_476', 'action_2572', 'action_1325', 'action_2677', 'action_2724', 'action_2636', 'action_2847', 'action_2797', 'action_2778', 'action_1380', 'action_1400', 'action_154

In [26]:
print(f"Only {len(referee_detected_videos)} out of {len(all_videos)} videos have a referee detected. ({len(referee_detected_videos)/len(all_videos)*100:.2f}%)")    

Only 224 out of 400 videos have a referee detected. (56.00%)


In [28]:
no_offence_actions = os.listdir("data/frames/no_offence")
offence_actions = os.listdir("data/frames/offence")


referee_detected_offence_actions = []
referee_detected_no_offence_actions = []
for referee_detected_video in referee_detected_videos:
    if referee_detected_video in no_offence_actions:
        referee_detected_no_offence_actions.append(referee_detected_video)
    else:
        referee_detected_offence_actions.append(referee_detected_video)

print(f"Referee detected in {len(referee_detected_offence_actions)} out of {len(offence_actions)} videos with offences")
print(f"Referee detected in {len(referee_detected_no_offence_actions)} out of {len(no_offence_actions)} videos without offences")

print(referee_detected_offence_actions[:3])
print(referee_detected_no_offence_actions[:3])


Referee detected in 112 out of 200 videos with offences
Referee detected in 112 out of 200 videos without offences
['action_2226', 'action_637', 'action_2508']
['action_2230', 'action_2725', 'action_1670']


In [12]:
from create_pip_video import create_pip_video

for action_name in referee_detected_videos:
    original_video_path = f"data/Dataset/Train/{action_name}/clip_0.mp4"
    minimap_video_path = f"visualization/output/{action_name}/video/{action_name}.mp4"


    if f"{action_name}.mp4" in os.listdir("data/pip_videos"):
        print(f"Skipping {action_name} because it already exists")
        continue

    create_pip_video(
        original_video_path, 
        minimap_video_path, 
        "data/pip_videos/" + action_name + ".mp4", 
        pip_scale=0.3
    )

Skipping action_2230 because it already exists
Skipping action_2725 because it already exists
Skipping action_1670 because it already exists
Skipping action_1959 because it already exists
Skipping action_2226 because it already exists
Skipping action_637 because it already exists
Skipping action_2508 because it already exists
Skipping action_1975 because it already exists
Skipping action_2089 because it already exists
Skipping action_477 because it already exists
Skipping action_2748 because it already exists
Skipping action_1871 because it already exists
Skipping action_1037 because it already exists
Skipping action_573 because it already exists
Skipping action_1963 because it already exists
Skipping action_748 because it already exists
Skipping action_461 because it already exists
Skipping action_2836 because it already exists
Skipping action_2904 because it already exists
Skipping action_2841 because it already exists
Skipping action_504 because it already exists
Skipping action_194

https://pmc.ncbi.nlm.nih.gov/articles/PMC7820113/



## Score the referee based on metrics


In [34]:
import numpy as np
import json

def referee_position_score(ref_pos, action_pos, d_diag, 
                           d_action_optimal=20, sigma_action=5,
                           sigma_diag=10):
    x_r, y_r = ref_pos
    x_a, y_a = action_pos
    d_action = np.linalg.norm([x_r - x_a, y_r - y_a])

    # Score for action proximity (peak at 20m, penalizes further/closer)
    score_action = np.exp(-((d_action - d_action_optimal)**2) / (2 * sigma_action**2))

    # Score for diagonal alignment (peak at 0m, falls off with distance)
    score_diag = np.exp(-(d_diag**2) / (2 * sigma_diag**2))

    # Combine both scores
    score_total = (score_action + score_diag) / 2

    return round(score_total, 3)

# Process all referee detected videos (or a subset for testing)
results = {}

for action_name in referee_detected_videos:
    video_info = json.load(open(f"data/processed_gamestate_output/{action_name}.json"))
    
    frame_scores = []
    
    for frame_id, frame_info in video_info.items():
        frame_info = FrameInfo.model_validate(frame_info)
        
        # Find the referee
        referee = None
        for player in frame_info.players:
            if player.role == 'referee':
                referee = player
                break
        
        if referee and frame_info.point_of_action and frame_info.dist_to_diagonal:
            # Get referee position
            ref_pos = (referee.position.x, referee.position.y)
            
            # Get point of action position
            action_pos = (frame_info.point_of_action.x, frame_info.point_of_action.y)
            
            # Get distance to diagonal
            d_diag = frame_info.dist_to_diagonal
            
            # Calculate score
            score = referee_position_score(ref_pos, action_pos, d_diag)
            frame_scores.append(score)
                
    
    # Calculate overall score for the clip
    if frame_scores:
        avg_score = round(sum(frame_scores) / len(frame_scores), 3)
        results[action_name] = {
            "num_frames": len(frame_scores),
            "avg_score": avg_score,
            "min_score": min(frame_scores),
            "max_score": max(frame_scores)
        }
    else:
        print(f"No referee detected in frames for {action_name}")

# Save overall results to a file
with open("referee_scores.json", "w") as f:
    json.dump(results, f, indent=2)

In [36]:
with open("referee_scores.json", "r") as f:
    results = json.load(f)

offence_results = {}
no_offence_results = {}
for action_name, result in results.items():
    if action_name in offence_actions:
        offence_results[action_name] = result
    elif action_name in no_offence_actions:
        no_offence_results[action_name] = result

print(len(offence_results))
print(len(no_offence_results))

112
112


In [39]:
# Remove outliers using IQR method
def remove_outliers(data_dict):
    avg_scores = [result["avg_score"] for result in data_dict.values()]
    
    # Calculate Q1, Q3 and IQR
    q1 = np.percentile(avg_scores, 25)
    q3 = np.percentile(avg_scores, 75)
    iqr = q3 - q1
    
    # Define bounds
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Filter out outliers
    filtered_dict = {
        action: result for action, result in data_dict.items()
        if lower_bound <= result["avg_score"] <= upper_bound
    }
    
    return filtered_dict

# Remove outliers from both sets
offence_results = remove_outliers(offence_results)
no_offence_results = remove_outliers(no_offence_results)

print("After removing outliers:")
print(f"Number of offence clips: {len(offence_results)}")
print(f"Number of no-offence clips: {len(no_offence_results)}")


After removing outliers:
Number of offence clips: 112
Number of no-offence clips: 112


In [40]:
# Calculate statistics for offence results
offence_avg_scores = [result["avg_score"] for result in offence_results.values()]
offence_min_scores = [result["min_score"] for result in offence_results.values()]
offence_max_scores = [result["max_score"] for result in offence_results.values()]

print("\nOffence Results Statistics:")
print(f"Average scores: mean={round(sum(offence_avg_scores)/len(offence_avg_scores), 3)}, min={min(offence_avg_scores)}, max={max(offence_avg_scores)}")
print(f"Minimum scores: mean={round(sum(offence_min_scores)/len(offence_min_scores), 3)}, min={min(offence_min_scores)}, max={max(offence_min_scores)}")
print(f"Maximum scores: mean={round(sum(offence_max_scores)/len(offence_max_scores), 3)}, min={min(offence_max_scores)}, max={max(offence_max_scores)}")

# Calculate statistics for no-offence results
no_offence_avg_scores = [result["avg_score"] for result in no_offence_results.values()]
no_offence_min_scores = [result["min_score"] for result in no_offence_results.values()]
no_offence_max_scores = [result["max_score"] for result in no_offence_results.values()]

print("\nNo Offence Results Statistics:")
print(f"Average scores: mean={round(sum(no_offence_avg_scores)/len(no_offence_avg_scores), 3)}, min={min(no_offence_avg_scores)}, max={max(no_offence_avg_scores)}")
print(f"Minimum scores: mean={round(sum(no_offence_min_scores)/len(no_offence_min_scores), 3)}, min={min(no_offence_min_scores)}, max={max(no_offence_min_scores)}")
print(f"Maximum scores: mean={round(sum(no_offence_max_scores)/len(no_offence_max_scores), 3)}, min={min(no_offence_max_scores)}, max={max(no_offence_max_scores)}")



Offence Results Statistics:
Average scores: mean=0.39, min=0.001, max=0.808
Minimum scores: mean=0.107, min=0.0, max=0.604
Maximum scores: mean=0.687, min=0.001, max=1.0

No Offence Results Statistics:
Average scores: mean=0.38, min=0.0, max=0.808
Minimum scores: mean=0.138, min=0.0, max=0.59
Maximum scores: mean=0.671, min=0.0, max=1.0
